# Evidence accumulation models
In this assignment, we simulate a simple *evidence accumulation model* from scratch. The following papers gives a general introduction to how evidence accumulation models (sometimes called 'sequential sampling models') are used:

- Forstmann, B. U., Ratcliff, R., & Wagenmakers, E.-J. (2016). Sequential Sampling Models in Cognitive Neuroscience: Advantages, Applications, and Extensions. Annual Review of Psychology, 67(1), 641–666. https://doi.org/10.1146/annurev-psych-122414-033645

The model we will simulate is described by Bogacz et al. (2006), p. 704, section 'DDM':

- Bogacz, R., Brown, E., Moehlis, J., Holmes, P., & Cohen, J. D. (2006). The physics of optimal decision making: A formal analysis of models of performance in two-alternative forced-choice tasks. Psychological Review, 113(4), 700–765. https://doi.org/10.1037/0033-295X.113.4.700


The model was originally proposed in (this is only intended as background literature, not required for this assigment):
- Stone, M. (1960). Models for choice-reaction time. Psychometrika, 25(3), 251–260. https://doi.org/10.1007/BF02289729
- Ratcliff, R. (1978). A theory of memory retrieval. Psychological Review, 85(2), 59–108.

Evidence accumulation models are formalized theories of decision making, often used to analyse data from two-alternative forced choice decision-making tasks. In this tasks, participants are presented with a stimulus, and are required to make a decision based on that stimulus. For example, the stimulus could be a Gabor patch tilted to the left or to the right:
<figure><img src="https://jspsychophysics.hes.kyushu-u.ac.jp/images/gabor.png"></figure>

and the participants need to respond 'left' or 'right'. Or the stimulus could be a [random-dot kinetogram](https://www.youtube.com/watch?v=7OdCe95IiLw), and the participants need to respond whether the cloud of dots moves to the left or to the right.


Evidence accumulation models assume that people make decisions by gradually accumulating evidence for each choice option while watching the stimulus. Evidence accumulation proceeds until a threshold level of evidence is reached. At that point, the decision corresponding to the threshold (sometimes called 'decision bound') is made. This is often visualized as follows:

<figure><img src='https://surfdrive.surf.nl/files/index.php/s/3ESvM7R9a5bs1LU/download'> </figure>

where the thresholds are indicated by the blue horizontal lines in the middle panel. The top threshold could indicate a choice for 'choose left', and the bottom threshold for 'choose right'. The gray line traces the evidence accumulation over time. In this example, evidence accumulation reaches the top threshold, and a choice for 'left' is made. If the stimulus indeed required a left answer, then this choice was correct.

By repeating this process over and over again across many trials, gradually, response time distributions arise:

<figure><img src='https://surfdrive.surf.nl/files/index.php/s/6pOmJXvpZzx4EDd/download'> </figure>

#### Structure of this assignment
In this assignment, we will gradually build up an evidence accumulation model in three steps:

1. We start by simulating a random walk;
2. We then end the random walk whenever a threshold is reached;
3. Then, we add a drift to the random walk

With this model, we then explore how the parameters are related to empirical data.

That covers the basics of an evidence accumulation model. With that simple model, we can simulate decision times, choices, and the effects of changing the parameters

In [2]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Part 1. Simulate a random walk

A random walk is characterized by the differential equation:

$dx/dt = cdW$

i.e., any change in $x$ during time step $dt$ is Gaussian noise $dW$ with variance $c* dt$. So essentially, at every time step, we just add normally distributed noise to $x$.

Another way of describing the this process is the recursive equation:

$x_{i+1} = x_{i} + cdW$ 


### Assignment 1.1
To simulate a random walk process, initiate variable `x` at 0. Then, create a `for`-loop with 5000 iterations. Every iteration of the `for`-loop approximates `dt` seconds, i.e., when we set `dt = 0.001`, we simulate with a 1 millisecond time resolution, and in total we simulate 5 seconds (5000 times 1 millisecond)

In every iteration of the `for`-loop, do the following:
 - increase `x` with Gaussian noise with standard deviation $\sqrt{c* dt}$ to `x`, where `c = 1` and `dt = 0.001`
 - save the state of `x` to a vector called `xs`

After all iterations, the vector `xs` should contain the full random walk over time.

(NB: the numpy function for random Gaussian noise requests the standard deviation, not the variance, so you need to calculate the standard deviation before passing it to the function)

In [3]:
np.random.seed(5)  # don't remove this -- this makes sure the random noise is the same every time you call this function

## your code here


#### Assignment 1.2
After the loop, plot `xs` to visualize the random walk across time. If you implement it correctly, it should look like this:      <figure>
      <img src="https://surfdrive.surf.nl/files/index.php/s/FFKigsGcHTiInUl/download" alt="Random walk" style="width:400pt">
      </figure>


In [4]:
## your code here

## Part 2. Stop the random walk when it reaches a threshold

Assume that evidence accumulation continues until `x` reaches either `+a` or `-a`. At that point, the random walk has reached a decision. The decision time is the number of iterations required to reach the threshold (keeping in mind the time resolution: so 1000 iterations corresponds to $1000 \times dt = 1000 \times 0.001 = 1 $ second )

#### Assignment 2.1
Set `a = 0.5` and simulate the for loop until `a` is reached (with maximally 5000 iterations). What was the decision time? And which choice was made (i.e., the upper bound of `+a` or the lower bound of `-a`)?

In [5]:
### your code


#### Assignment 2.2. Wrap your code in a function
Create a new function called `simulate_single_trial`. It takes as arguments: `a`, `c`, `dt`, and `max_time_steps`. In the function, the for loop is executed. The function returns the decision time, the choice, and `xs`

In [6]:
### your code

The function provides a conventient way to simulate multiple trials.

#### Assignment 2.3
Simulate 500 trials. After every trial, save the decision time and the choice.
What is the mean decision time across the 500 trials? And what was the error rate (the proportion of trials reaching the *lower* bound?)

In [7]:
### your code

#### Assignment 2.4
Plot the [histogram](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html) of decision times

In [9]:
### your code

## Part 3. Add a drift. 

The differential equation for a random walk with drift is:

$dx/dt = vdt + cdW$

Where `v` is known as the drift rate.

#### Assignment 3.1
Implement a drift rate in the `for` loop of your function `simulate_single_trial`. Note that drift rate `v` needs to be scaled (multiplied) by `dt` before adding it to `x`!

In [10]:
### your code

#### Assignment 3.2
Again, simulate 500 trials. Use a drift rate of `v=1`. After every trial, store the decision time and the choice. What is the mean decision time across trials? What was the error rate?

In [11]:
### your code

It's possible to calculate the mean response time and the error rate analytically. Bogacz et al (2006) give the following Equations 8:

In [12]:
def ER(v, a, c):
    ''' Returns error rate (proportion of errors) given drift rate `v`, threshold `a`, and diffusion noise `c`'''
    return 1/(1+np.exp(2*v*a/c**2))
    
def DT(v, a, c):
    ''' Returns mean decision time given drift rate `v`, threshold `a`, and diffusion noise `c`'''
    if v == 0:
        ### prevent division by 0
        v = 0.00000001
    return (a/v) * np.tanh((v*a)/(c**2))

#### Assignment 3.3
Check whether your simulated mean RT and error rate approximate to the analytical mean RT and error rate. Note that due to simulation noise, small deviations from the analytical values can be expected. The more trials you simulate, the closer the analytical and simulated values should get.

In [13]:
### your code

## Part 4. The effects of the drift and threshold parameters
In this section, we explore what the effect is of increasing/decreasing the drift rate on the simulated data. Similarly, we explore what the effect is of increasing/decreasing the threshold on the simulated data

#### Assignment 4.1
Simulate one dataset of 500 trials with `v=1` and `a=1`, and another dataset with `v=2` and `a=1`. Compare the mean decision time, the error rate, and the histograms of the two datasets.

In [14]:
# your code

#### Assignment 4.2
Simulate one dataset of 500 trials with `v=1` and `a=1`, and another dataset with `v=1` and `a=1.5`. Compare the mean decision time, the error rate, and the histograms of the two datasets.

In [15]:
# your code